In [ ]:
## 1. Create Dataframe {subject}_{sesssion}

# Import Libraries

In [26]:
import pandas as pd
import mne
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import signal
from itertools import groupby
import math
import neurokit2 as nk
from datetime import datetime
import json
import re
from scipy.stats import linregress
import os 
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score, multilabel_confusion_matrix
from sklearn.preprocessing import StandardScaler
from scipy.signal import butter, lfilter
import time


# https://neuropsychology.github.io/NeuroKit/functions/eog.html

# I. Preparing Data

In [15]:
subject_num = 9
file_path = f'data_analysis/data_BlinkGaze/sub{subject_num}s0/{subject_num}_data_blinkGaze.csv'
data_blinkGaze = pd.read_csv(file_path)

file_path = f'data_analysis/data_BlinkGaze/sub{subject_num}s0/{subject_num}_data_wordBlink.csv'
data_wordBlink= pd.read_csv(file_path)

file_path = f'data_analysis/data_BlinkGaze/sub{subject_num}s0/{subject_num}_data_cardGame.csv'
data_cardGame = pd.read_csv(file_path)

#This missing for subject 9 
file_path = f'data_analysis/data_BlinkGaze/sub{subject_num}s0/{subject_num}_data_CG_Recall.csv'
data_CGRecall = pd.read_csv(file_path)

file_path = f'data_analysis/data_BlinkGaze/sub{subject_num}s0/{subject_num}_data_WB_Recall.csv'
data_WBRecall = pd.read_csv(file_path)


file_path = f'data_analysis/data_BlinkGaze/sub{subject_num}s0/{subject_num}_label_blinkGaze.csv'
label_blinkGaze = pd.read_csv(file_path)

file_path = f'data_analysis/data_BlinkGaze/sub{subject_num}s0/{subject_num}_label_wordBlink.csv'
label_wordBlink= pd.read_csv(file_path)

file_path = f'data_analysis/data_BlinkGaze/sub{subject_num}s0/{subject_num}_label_cardGame.csv'
label_cardGame = pd.read_csv(file_path)

# This missing for subject 9 
file_path = f'data_analysis/data_BlinkGaze/sub{subject_num}s0/{subject_num}_label_CG_Recall.csv'
label_CGRecall = pd.read_csv(file_path)

file_path = f'data_analysis/data_BlinkGaze/sub{subject_num}s0/{subject_num}_label_WB_Recall.csv'
label_WBRecall = pd.read_csv(file_path)

In [16]:
# View the content of data (Contains eeg0, eeg1, eog0)
data_blinkGaze

,timestamp,eeg0,eeg1,eog0,time_seconds,blinkLabel,gazeLRLabel,gazeLeftLabel,gazeRightLabel,restLabel
0,1712514341489,-158982.132653,-169375.885951,-107705.597104,0.117,1,0,0,0,0
1,1712514341493,-158974.407217,-169398.243646,-107691.958675,0.121,1,0,0,0,0
2,1712514341497,-158969.681781,-169421.601340,-107677.320246,0.125,1,0,0,0,0
3,1712514341501,-158977.956345,-169415.959034,-107685.681817,0.129,1,0,0,0,0
4,1712514341505,-158988.230909,-169400.316728,-107701.043388,0.133,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
213603,1712515983037,-60392.674610,-75363.910127,-150456.587030,1641.665,0,0,0,0,0
213604,1712515983041,-60226.949174,-75560.267821,-150461.948601,1641.669,0,0,0,0,0
213605,1712515983045,-60484.223738,-75585.625515,-150305.310172,1641.673,0,0,0,0,0
213606,1712515983049,-60574.498302,-75566.983210,-150282.671742,1641.677,0,0,0,0,0


In [17]:
# View the content of label (Contains instruction)
label_blinkGaze

,timestamp,time_seconds,instruction,blink_instruction,gazeLeft_instruction,gazeRight_instruction,gazeLR_instruction,rest_instruction
0,1712514341372,0.000,NaN,0,0,0,0,0
1,1712514341372,0.000,lightblink,1,0,0,0,0
2,1712514341372,0.000,NaN,0,0,0,0,0
3,1712514342066,0.694,white,0,0,0,0,0
4,1712514346064,4.692,lightblink,1,0,0,0,0
...,...,...,...,...,...,...,...,...
634,1712515962059,1620.687,longblink,1,0,0,0,0
635,1712515963056,1621.684,white,0,0,0,0,0
636,1712515967059,1625.687,gazeleft,0,1,0,1,0
637,1712515968058,1626.686,white,0,0,0,0,0


In [18]:
# The data for Phase 1 blinkGaze is segmented by "rest", so we will cut the data and save each sequence to series_cuts_labels_blinkGazeStudy
def cut_series_by_rest_indices(data, rest_indices):
    cuts = []
    start_index = 0
    
    for end_index in rest_indices:
        if (end_index-start_index > 2):
            cuts.append(data[start_index:end_index])
        start_index = end_index + 1
    
    if start_index < len(data):
        cuts.append(data[start_index:])
    
    return cuts

In [20]:
rest_indices_labels_blinkGazeStudy = label_blinkGaze[label_blinkGaze["rest_instruction"] ==1].index
series_cuts_labels_blinkGazeStudy = cut_series_by_rest_indices(label_blinkGaze, rest_indices_labels_blinkGazeStudy)

# View the first label sequence element cut by rest 
series_cuts_labels_blinkGazeStudy[1]

,timestamp,time_seconds,instruction,blink_instruction,gazeLeft_instruction,gazeRight_instruction,gazeLR_instruction,rest_instruction
93,1712514526055,184.683,longblink,1,0,0,0,0
94,1712514527058,185.686,white,0,0,0,0,0
95,1712514531054,189.682,longblink,1,0,0,0,0
96,1712514532055,190.683,white,0,0,0,0,0
97,1712514534054,192.682,longblink,1,0,0,0,0
...,...,...,...,...,...,...,...,...
178,1712514697054,355.682,white,0,0,0,0,0
179,1712514701055,359.683,longblink,1,0,0,0,0
180,1712514702053,360.681,white,0,0,0,0,0
181,1712514705055,363.683,longblink,1,0,0,0,0


In [23]:
rest_indices_data_blinkGazeStudy = data_blinkGaze[data_blinkGaze["restLabel"] ==1].index
series_cuts_data_blinkGazeStudy = cut_series_by_rest_indices(data_blinkGaze, rest_indices_data_blinkGazeStudy)

# View the first data sequence element cut by rest 
series_cuts_data_blinkGazeStudy[1]

,timestamp,eeg0,eeg1,eog0,time_seconds,blinkLabel,gazeLRLabel,gazeLeftLabel,gazeRightLabel,restLabel
22700,1712514516225,-239329.191292,-247507.396672,-193879.459553,174.853,0,0,0,0,0
22701,1712514516229,-239312.465856,-247552.754366,-193831.821124,174.857,0,0,0,0,0
22702,1712514516233,-239309.740420,-247596.112060,-193815.182695,174.861,0,0,0,0,0
22703,1712514516237,-239310.014984,-247586.469754,-193821.544266,174.865,0,0,0,0,0
22704,1712514516241,-239315.289548,-247553.827449,-193843.905837,174.869,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
47785,1712514708133,-298281.953661,-309798.488327,-265185.545803,366.761,0,0,0,0,0
47786,1712514708137,-298276.228226,-309791.846021,-265196.907374,366.765,0,0,0,0,0
47787,1712514708141,-298311.502790,-309737.203715,-265197.268944,366.769,0,0,0,0,0
47788,1712514708145,-298314.777354,-309730.561409,-265231.630515,366.773,0,0,0,0,0


In [25]:
sequence_lengths = []

# Iterate over each DataFrame in the array
for df_sequence in series_cuts_labels_blinkGazeStudy:
    sequence_length = len(df_sequence)  # Length of the DataFrame, which represents the sequence
    sequence_lengths.append(sequence_length)  # Append the length to the list

# Calculate min, max, and average lengths
min_length = min(sequence_lengths)
max_length = max(sequence_lengths)
average_length = sum(sequence_lengths) / len(sequence_lengths)

print("Minimum sequence length:", min_length, "data points")
print("Maximum sequence length:", max_length, "data points")
print("Average sequence length:", average_length, "data points")

Minimum sequence length: 40 data points
Maximum sequence length: 96 data points
Average sequence length: 57.09090909090909 data points


# II. Hyperparameter Tuning

In [ ]:
def extract_features(window, lowcut, highcut):
    # Example: Apply your butterworth filter to the window
    sr =250
    filtered_window = np.apply_along_axis(signal_filter_butterworth, 0, window, sr, lowcut, highcut) #1.5 15
    # Example: Calculate mean and standard deviation of the filtered window
    mean_values = np.mean(filtered_window, axis=0)
    std_values = np.std(filtered_window, axis=0)
    return np.concatenate([mean_values, std_values])


def hyperparameter_tuning_ML_new3(sequences, signals, window_start, window_end, lowcut_start, lowcut_end, highcut_start, highcut_end, metric='f1'):
    if metric not in ['accuracy', 'f1', 'auc']:
        raise ValueError("Invalid metric. Choose 'accuracy', 'f1', or 'auc'.")

    start_time = time.time()
    best_metric = 0
    best_combination = None
    best_report = None
    step_size = 250

    window_range = list(range(window_start, window_end, 100))
    lowcut_range = [lowcut_start + i * 0.2 for i in range(int((lowcut_end - lowcut_start) / 0.2) + 1)]
    highcut_range = list(range(highcut_start, highcut_end, 2))
    rounds = len(window_range) * len(lowcut_range) * len(highcut_range)
    print(f"We will run {rounds} rounds of fitting for all the combinations of hyperparameters given.")

    # Lists to store results for plotting
    lowcut_results = []
    highcut_results = []
    window_results = []
    metric_results = []
    
    for window_size, lowcut, highcut in product(window_range, lowcut_range, highcut_range):
        features = []
        labels = []
        
        for sequence in sequences:
            for i in range(0, len(sequence) - window_size, step_size):
                window = sequence.iloc[i:i+window_size]
                label = window.iloc[0]["blinkLabel"] 
                #label = window.iloc[0]["gazeLeftLabel", "gazeRightLabel", "blinkLabel", "gazeLRLabel"]  # Assuming label is in "blinkLabel" column
                window_signals = window[signals].values  # Extract features based on signals
                features.append(window_signals)  # Keep the features as windows
                labels.append(label)
    
        features = np.array(features)
        labels = np.array(labels)

        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=42)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
        X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

        clf = RandomForestClassifier(n_estimators=50, random_state=42)
        clf.fit(X_train.reshape(X_train.shape[0], -1), y_train)

        y_pred = clf.predict(X_test.reshape(X_test.shape[0], -1))

        if metric == 'accuracy':
            current_metric = accuracy_score(y_test, y_pred)
        elif metric == 'f1':
            current_metric = f1_score(y_test, y_pred, average='macro')  # Micro-F1 for multilabel
        elif metric == 'auc':
            # AUC is not directly applicable to multilabel classification
            current_metric = None

        # Calculate F1-score for each label
        label_f1_scores = f1_score(y_test, y_pred, average= None)

        report = classification_report(y_test, y_pred, zero_division=0)

        # Append results for plotting
        lowcut_results.append(lowcut)
        highcut_results.append(highcut)
        window_results.append(window_size)
        metric_results.append(current_metric)

        if current_metric is not None and current_metric > best_metric:
            best_metric = current_metric
            best_combination = (window_size, lowcut, highcut)
            best_report = report

        print(f"Current combination: {(window_size, lowcut, highcut)}")
        print(f"Current {metric}: {current_metric}")
        print(f"F1-score for each label: {label_f1_scores}")
        print(f"Best combination: {best_combination}")
        print(f"Best {metric}: {best_metric}")
        print(best_report)
        print("____________________________")
    
    end_time = time.time()  # End time measurement
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print("#############################")
    print("Best combination:", best_combination)
    print(f"Best {metric}:", best_metric)
    print(best_report)
    print(f"Elapsed Time: {elapsed_time}")

    return lowcut_results, highcut_results, window_results, metric_results, best_combination, best_metric, best_report


In [ ]:
def hyperparameter_tuning_ML_multiclass(sequences, signals, window_start, window_end, lowcut_start, lowcut_end, highcut_start, highcut_end, metric='f1'):
    if metric not in ['accuracy', 'f1', 'auc']:
        raise ValueError("Invalid metric. Choose 'accuracy', 'f1', or 'auc'.")

    start_time = time.time()
    best_metric = 0
    best_combination = None
    best_report = None
    step_size = 250

    window_range = list(range(window_start, window_end, 100))
    lowcut_range = [lowcut_start + i * 0.5 for i in range(int((lowcut_end - lowcut_start) / 0.5) + 1)]
    highcut_range = list(range(highcut_start, highcut_end, 2))
    rounds = len(window_range) * len(lowcut_range) * len(highcut_range)
    print(f"We will run {rounds} rounds of fitting for all the combinations of hyperparameters given.")

    # Lists to store results for plotting
    lowcut_results = []
    highcut_results = []
    window_results = []
    metric_results = []

    for window_size, lowcut, highcut in product(window_range, lowcut_range, highcut_range):
        features = []
        labels = []

        for sequence in sequences:
            for i in range(0, len(sequence) - window_size, step_size):
                window = sequence.iloc[i:i+window_size]
                # Extract all three labels
                label = window[['gazeLRLabel', 'blinkLabel']].values.flatten()
                window_signals = window[signals].values  # Extract features based on signals
                features.append(window_signals)  # Keep the features as windows
                labels.append(label)

        features = np.array(features)
        labels = np.array(labels)

        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=42)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
        X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

        clf = RandomForestClassifier(n_estimators=50, random_state=42)
        clf.fit(X_train.reshape(X_train.shape[0], -1), y_train)

        y_pred = clf.predict(X_test.reshape(X_test.shape[0], -1))

        if metric == 'accuracy':
            current_metric = accuracy_score(y_test, y_pred)
        elif metric == 'f1':
            current_metric = f1_score(y_test, y_pred, average='macro')  # Weighted F1-score for multiclass
        elif metric == 'auc':
            # AUC is not directly applicable to multiclass classification
            current_metric = None

        report = classification_report(y_test, y_pred)  # No need to specify average parameter for multiclass
        print(report)
        # Append results for plotting
        lowcut_results.append(lowcut)
        highcut_results.append(highcut)
        window_results.append(window_size)
        metric_results.append(current_metric)

        if current_metric is not None and current_metric > best_metric:
            best_metric = current_metric
            best_combination = (window_size, lowcut, highcut)
            best_report = report

        print(f"Current combination: {(window_size, lowcut, highcut)}")
        print(f"Current {metric}: {current_metric}")
        print(f"F1-score for each class:")
        print(f"Best combination: {best_combination}")
        print(f"Best {metric}: {best_metric}")
        print("____________________________")

    end_time = time.time()  # End time measurement
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print("#############################")
    print("Best combination:", best_combination)
    print(f"Best {metric}:", best_metric)
    print(best_report)
    print(f"Elapsed Time: {elapsed_time}")

    return lowcut_results, highcut_results, window_results, metric_results, best_combination, best_metric, best_report


In [ ]:
def plot_results_avg(lowcut_results, highcut_results, window_results, accuracy_results):
    fig, axs = plt.subplots(3, 1, figsize=(10, 15))

    # Calculate average accuracy for each unique lowcut value
    lowcut_avg_accuracy = []
    unique_lowcut_values = np.unique(lowcut_results)
    max_lowcut_avg = -np.inf
    max_lowcut_value = None
    for lowcut_value in unique_lowcut_values:
        indices = np.where(lowcut_results == lowcut_value)
        avg_accuracy = np.mean(np.array(accuracy_results)[indices])
        lowcut_avg_accuracy.append(avg_accuracy)
        if avg_accuracy > max_lowcut_avg:
            max_lowcut_avg = avg_accuracy
            max_lowcut_value = lowcut_value
    
    axs[0].plot(unique_lowcut_values, lowcut_avg_accuracy, marker='o')
    axs[0].set_title('Average Accuracy vs. Lowcut Value')
    axs[0].set_xlabel('Lowcut Value')
    axs[0].set_ylabel('Average Accuracy')

    # Calculate average accuracy for each unique highcut value
    highcut_avg_accuracy = []
    unique_highcut_values = np.unique(highcut_results)
    max_highcut_avg = -np.inf
    max_highcut_value = None
    for highcut_value in unique_highcut_values:
        indices = np.where(highcut_results == highcut_value)
        avg_accuracy = np.mean(np.array(accuracy_results)[indices])
        highcut_avg_accuracy.append(avg_accuracy)
        if avg_accuracy > max_highcut_avg:
            max_highcut_avg = avg_accuracy
            max_highcut_value = highcut_value
    
    axs[1].plot(unique_highcut_values, highcut_avg_accuracy, marker='o')
    axs[1].set_title('Average Accuracy vs. Highcut Value')
    axs[1].set_xlabel('Highcut Value')
    axs[1].set_ylabel('Average Accuracy')

    # Calculate average accuracy for each unique window size value
    window_avg_accuracy = []
    unique_window_values = np.unique(window_results)
    max_window_avg = -np.inf
    max_window_value = None
    for window_value in unique_window_values:
        indices = np.where(window_results == window_value)
        avg_accuracy = np.mean(np.array(accuracy_results)[indices])
        window_avg_accuracy.append(avg_accuracy)
        if avg_accuracy > max_window_avg:
            max_window_avg = avg_accuracy
            max_window_value = window_value
    
    axs[2].plot(unique_window_values, window_avg_accuracy, marker='o')
    axs[2].set_title('Average Accuracy vs. Window Size')
    axs[2].set_xlabel('Window Size')
    axs[2].set_ylabel('Average Accuracy')

    plt.tight_layout()
    plt.show()

    print(f"Maximum Average Accuracy for Lowcut: {max_lowcut_avg} (Lowcut value: {max_lowcut_value})")
    print(f"Maximum Average Accuracy for Highcut: {max_highcut_avg} (Highcut value: {max_highcut_value})")
    print(f"Maximum Average Accuracy for Window Size: {max_window_avg} (Window size: {max_window_value})")

    return max_lowcut_value, max_highcut_value, max_window_value

    
def plot_results(lowcut_results, highcut_results, window_results, accuracy_results):
    fig, axs = plt.subplots(3, 1, figsize=(10, 15))

    axs[0].plot(lowcut_results, accuracy_results, marker='o', linestyle='-')
    axs[0].set_title('Accuracy vs. Lowcut Value')
    axs[0].set_xlabel('Lowcut Value')
    axs[0].set_ylabel('Accuracy')

    axs[1].plot(highcut_results, accuracy_results, marker='o', linestyle='-')
    axs[1].set_title('Accuracy vs. Highcut Value')
    axs[1].set_xlabel('Highcut Value')
    axs[1].set_ylabel('Accuracy')

    axs[2].plot(window_results, accuracy_results, marker='o', linestyle='-')
    axs[2].set_title('Accuracy vs. Window Size')
    axs[2].set_xlabel('Window Size')
    axs[2].set_ylabel('Accuracy')

    plt.tight_layout()
    plt.show()


In [ ]:
#prepared_data_channels_blinkGazeStudy_masked = mask_dataframe_intervals(prepared_data_channels_blinkGazeStudy, mask_intervals_rest)#
#prepared_data_channels_blinkGazeStudy_masked 

In [ ]:
## EOG Hyperparameter Tuning    
#0.5-2.5 seconds

# Call the hyperparameter tuning function and get the result
lowcut_results, highcut_results, window_results, metric_results, best_combination,  best_metric, best_report = hyperparameter_tuning_ML_new3(time_series_cuts_data_channels_blinkGazeStudy, ['eeg0', 'eeg1', 'eog0'], 500, 1000, 0.5, 5, 10, 40, 'f1')

In [ ]:
window = best_combination[0]
window_size = best_combination[0]
lowcut = best_combination[1]
highcut= best_combination[2]
print(f"window: {window}, lowcut: {lowcut}, highcut:{highcut}")

In [ ]:
max_lowcut, max_highcut, max_window = plot_results_avg(lowcut_results, highcut_results, window_results, metric_results)

In [ ]:
## EOG Hyperparameter Tuning    
#0.5-2.5 seconds

# Call the hyperparameter tuning function and get the result
lowcut_results4, highcut_results4, window_results4, metric_results4, best_combination4,  best_metric4, best_report4 =  hyperparameter_tuning_ML_new3(time_series_cuts_data_channels_blinkGazeStudy, ['eeg0', 'eeg1', 'eog0'], 250, 1000, 0.1, 4, 5, 20, 'f1')


In [ ]:
window4 = best_combination4[0]
window_size4 = best_combination4[0]
lowcut4 = best_combination4[1]
highcut4= best_combination4[2]
print(f"window: {window4}, lowcut: {lowcut4}, highcut:{highcut4}")

In [ ]:
lowcut_results2, highcut_results2, window_results2, metric_results2, best_combination2, best_metric2, best_report2 = hyperparameter_tuning_ML_new_mask(prepared_data_channels_blinkGazeStudy, ['eeg0', 'eeg1'], 500, 1000, 0.5, 5, 10, 40, mask_intervals_rest, 'f1')

In [ ]:
window = best_combination2[0]
window_size = best_combination2[0]
lowcut = best_combination2[1]
highcut= best_combination2[2]
print(f"window: {window}, lowcut: {lowcut}, highcut:{highcut}")
print(best_metric2)
print(best_report2)

In [ ]:
max_lowcut2, max_highcut2, max_window2 = plot_results_avg(lowcut_results2, highcut_results2, window_results2, metric_results2)

In [ ]:
##hyperparameter_tuning_ML_new_all],250, 750, 0.5, 3, 10, 40,
lowcut_results3, highcut_results3, window_results3, metric_results3, best_combination3, best_metric3, best_report3 = hyperparameter_tuning_ML_new_mask(prepared_data_channels_blinkGazeStudy, [ 'eog0'], 500, 1000, 0.5, 5, 10, 40, mask_intervals_rest, 'f1')

#,100, 900, 0.5, 3, 15, 35, 'accuracy')

In [ ]:
window = best_combination3[0]
window_size = best_combination3[0]
lowcut = best_combination3[1]
highcut= best_combination3[2]
print(f"window: {window}, lowcut: {lowcut}, highcut:{highcut}")
print(best_metric3)
print(best_report3)

In [ ]:
max_lowcut3, max_highcut3, max_window3 = plot_results_avg(lowcut_results3, highcut_results3, window_results3, metric_results3)

In [ ]:

#lowcut_results4, highcut_results4, window_results4, metric_results4, best_combination4,  best_metric4, best_report4 = hyperparameter_tuning_ML_new_mask(prepared_data_channels_blinkGazeStudy, ['eeg0', 'eeg1', 'eog0'], 100, 1000, 0.5, 3, 10, 40, mask_intervals_rest, 'accuracy')

In [ ]:
#lowcut_results5, highcut_results5, window_results5, metric_results5, best_combination5, best_metric5, best_report5 = hyperparameter_tuning_ML_new_mask(prepared_data_channels_blinkGazeStudy, ['eeg0', 'eeg1'], 100, 1000, 0.5, 3, 10, 40, mask_intervals_rest, 'accuracy')

In [ ]:
#lowcut_results6, highcut_results6, window_results6, metric_results6, best_combination6, best_metric6, best_report6 = hyperparameter_tuning_ML_new_mask(prepared_data_channels_blinkGazeStudy, [ 'eog0'], 100, 1000, 0.5, 3, 10, 40, mask_intervals_rest, 'f1')

# III. Modeling

## A. Using Two files

In [ ]:
window = best_combination[0]
window_size = best_combination[0]
lowcut = best_combination[1]
highcut= best_combination[2]
print(f"window: {window}, lowcut: {lowcut}, highcut:{highcut}")


# window = max_window
# window_size = max_window
# lowcut =max_lowcut
# highcut= max_highcut
# print(f"window: {window}, lowcut: {lowcut}, highcut:{highcut}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.signal import butter, lfilter

sr = 250
step_size = 250# Adjust based on your data

# Assuming you have your data in DataFrames named prepared_data_channels and prepared_data_channels2
# And assuming you have labels in the column "blinkLabel" and features in column "eog0"

# Concatenate the two dataframes
combined_data = pd.concat([prepared_data_channels, prepared_data_channels2], ignore_index=True)


# Apply sliding window and extract features
features = []
labels = []

for i in range(0, len(combined_data) - window_size, step_size):
    window = combined_data.loc[:, ['eog0']].iloc[i:i+window_size-1, :]
    label = combined_data["blinkLabel"].iloc[i]
    feature_vector = extract_features(window, lowcut, highcut)
    features.append(feature_vector)
    labels.append(label)

# Convert lists to arrays
features = np.array(features)
labels = np.array(labels)

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(features)

# Split data into train and test sets
X_train, X_test, labels_train, labels_test = train_test_split(X, labels, test_size=0.5, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, labels_train)

# Predict on the test set
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

# Evaluate the model on the testing set
accuracy = accuracy_score(labels_test, y_pred)
f1 = f1_score(labels_test, y_pred)
auc = roc_auc_score(labels_test, y_prob[:, 1])
report = classification_report(labels_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(f'F1-score: {f1:.2f}')
print(f'AUC: {auc:.2f}')
print('Classification Report:\n', report)


## B. Using one file 

In [ ]:
window = best_combination[0]
window_size = best_combination[0]
lowcut = best_combination[1]
highcut= best_combination[2]
print(f"window: {window}, lowcut: {lowcut}, highcut:{highcut}")
print(best_report)

In [ ]:
window = max_window
window_size = max_window
lowcut =max_lowcut
highcut= max_highcut
print(f"window: {window}, lowcut: {lowcut}, highcut:{highcut}")

## i) Multi-class

In [ ]:
sr = 250

# Function to apply Butterworth filter
def signal_filter_butterworth(data, sr, lowcut, highcut):
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    order = 5
    b, a = butter(order, [low, high], btype='band')
    filtered_data = lfilter(b, a, data)
    return filtered_data

# Function to extract features from a window
def extract_features(window, lowcut, highcut):
    filtered_window = np.apply_along_axis(signal_filter_butterworth, 0, window, sr, lowcut, highcut)
    mean_values = np.mean(filtered_window, axis=0)
    std_values = np.std(filtered_window, axis=0)
    return np.concatenate([mean_values, std_values])

# Apply sliding window and extract features
features = []
labels = []
step_size = 250 

df = prepared_data_channels

for i in range(0, len(df) - window_size, step_size):
    window = df.loc[i:i+window_size-1, ['eog0', 'eeg0', 'eeg1']].values
    label = df.loc[i, ["blinkLabel", "gazeLeftLabel", "gazeRightLabel"]].values
    feature_vector = extract_features(window, lowcut, highcut)
    features.append(feature_vector)
    labels.append(label)

# Convert lists to arrays
features = np.array(features)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=20)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

# Evaluate the model on the testing set
accuracy = accuracy_score(y_test, y_pred)
#f1_micro = f1_score(y_test, y_pred, average='micro')
#auc_micro = roc_auc_score(y_test, y_prob, average='micro')
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
#print(f'F1-score (micro): {f1_micro:.2f}')
#print(f'AUC (micro): {auc_micro:.2f}')
print('Classification Report:\n', report)


## ii) One-class

In [ ]:
window = best_combination3[0]
window_size = window
lowcut = best_combination3[1]
highcut= best_combination3[2]
print(f"window: {window}, lowcut: {lowcut}, highcut:{highcut}")
print(best_report)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from scipy.signal import butter, lfilter

sr = 250
#window = 980
#window_size= 980
#lowcut = 1.5
#highcut= 15

# Function to extract features from a window
def extract_features(window, lowcut, highcut):
    # Example: Apply your butterworth filter to the window
    filtered_window = np.apply_along_axis(signal_filter_butterworth, 0, window, sr, lowcut, highcut)#sr=250, lowcut=1.5, highcut=15)
    
    # Example: Calculate mean and standard deviation of the filtered window
    mean_values = np.mean(filtered_window, axis=0)
    std_values = np.std(filtered_window, axis=0)
    
    return np.concatenate([mean_values, std_values])

# Apply sliding window and extract features
features = []
labels = []
step_size = 250 

df = prepared_data_channels

for i in range(0, len(df) - window_size, step_size):
    window = df.loc[:, [ 'eog0']].iloc[i:i+window_size-1, :]
    label = df["blinkLabel"].iloc[i]  # Assuming the label is the same for the entire window
    feature_vector = extract_features(window, lowcut, highcut)
    features.append(feature_vector)
    labels.append(label)

# Convert lists to arrays
features = np.array(features)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=20)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

# Evaluate the model on the testing set
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob[:, 1])
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(f'F1-score: {f1:.2f}')
print(f'AUC: {auc:.2f}')
print('Classification Report:\n', report)
